In [107]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

data = pd.read_csv("../bank.csv") 
# Decision Tree CLassifier

# A partir del E.D.A realizado en la regr.logistica, actualizar dataset 

In [108]:
columnas_a_usar= ["age", "balance", "day", "duration", #numericas
                  "job", "marital", "education", "contact", #categoricas
                  "deposit"] #variable a predecir

data_actualizado = data[columnas_a_usar]

#convertir la columna a predecir en binaria (numerica) 
data_actualizado['deposit'] = data_actualizado['deposit'].apply(lambda x: 1 if x == 'yes' else 0)

data[columnas_a_usar].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   balance    11162 non-null  int64 
 2   day        11162 non-null  int64 
 3   duration   11162 non-null  int64 
 4   job        11162 non-null  object
 5   marital    11162 non-null  object
 6   education  11162 non-null  object
 7   contact    11162 non-null  object
 8   deposit    11162 non-null  object
dtypes: int64(4), object(5)
memory usage: 785.0+ KB


/var/folders/h1/j3hlf_9d2wv_fdvmlsz9t0_m0000gn/T/ipykernel_25008/3540899115.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_actualizado['deposit'] = data_actualizado['deposit'].apply(lambda x: 1 if x == 'yes' else 0)


In [109]:
data_actualizado.head(2)

,age,balance,day,duration,job,marital,education,contact,deposit
0,59,2343,5,1042,admin.,married,secondary,unknown,1
1,56,45,5,1467,admin.,married,secondary,unknown,1


In [110]:
# Distribución de las variables numéricas
data_actualizado[["age", "balance", "day", "duration"]].describe()

,age,balance,day,duration
count,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818
std,11.913369,3225.413326,8.420740,347.128386
min,18.000000,-6847.000000,1.000000,2.000000
25%,32.000000,122.000000,8.000000,138.000000
50%,39.000000,550.000000,15.000000,255.000000
75%,49.000000,1708.000000,22.000000,496.000000
max,95.000000,81204.000000,31.000000,3881.000000


# Dividir el data set y Train Test Split

In [111]:
X = data_actualizado.drop('deposit', axis=1)
y = data_actualizado['deposit']

In [112]:
#dividir el dataset en train, val y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [113]:
import numpy as np
# Identificar columnas no numéricas en X_train
non_numeric_columns_train = X_train.select_dtypes(exclude=[np.number]).columns
print("Columnas no numéricas en X_train:", non_numeric_columns_train)


# Identificar columnas no numéricas en X_val
non_numeric_columns_val = X_val.select_dtypes(exclude=[np.number]).columns
print("Columnas no numéricas en X_val:", non_numeric_columns_val)

Columnas no numéricas en X_train: Index(['job', 'marital', 'education', 'contact'], dtype='object')
Columnas no numéricas en X_val: Index(['job', 'marital', 'education', 'contact'], dtype='object')


In [114]:
for col in non_numeric_columns_train:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')

for col in non_numeric_columns_val:
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce')

In [115]:
#chequear si hay nan en las columnas categoricas
X_train.isna().sum()

age             0
balance         0
day             0
duration        0
job          6696
marital      6696
education    6696
contact      6696
dtype: int64

In [116]:
#en que fila hay nan
X_train[X_train.isna().any(axis=1)]

,age,balance,day,duration,job,marital,education,contact
5613,57,1350,25,131,NaN,NaN,NaN,NaN
5988,31,530,18,251,NaN,NaN,NaN,NaN
5952,43,0,25,152,NaN,NaN,NaN,NaN
1001,32,6402,20,628,NaN,NaN,NaN,NaN
7916,52,297,7,55,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3607,27,861,14,212,NaN,NaN,NaN,NaN
6680,35,786,25,322,NaN,NaN,NaN,NaN
85,27,3706,16,897,NaN,NaN,NaN,NaN
10494,57,123,2,154,NaN,NaN,NaN,NaN


In [117]:
#chequear si hay nan
np.isnan(X_train_imputed).sum()

np.int64(0)

In [118]:
data_actualizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   balance    11162 non-null  int64 
 2   day        11162 non-null  int64 
 3   duration   11162 non-null  int64 
 4   job        11162 non-null  object
 5   marital    11162 non-null  object
 6   education  11162 non-null  object
 7   contact    11162 non-null  object
 8   deposit    11162 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 785.0+ KB


In [122]:
#chequear las columnas del data set data_actualizado
data_actualizado.columns

Index(['age', 'balance', 'day', 'duration', 'job', 'marital', 'education',
       'contact', 'deposit'],
      dtype='object')

In [125]:
from sklearn.tree import DecisionTreeClassifier

# Crear el modelo
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Predecir

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Train accuracy:", train_accuracy)
print("Validation accuracy:", val_accuracy)
print("Test accuracy:", test_accuracy)

ValueError: could not convert string to float: 'retired'

In [126]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_test_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

NameError: name 'y_test_pred' is not defined